In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import datetime
import geocoder
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',50)
pd.set_option('max_colwidth',200)
%matplotlib inline
import seaborn as sns

In [3]:
firm_2007 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2007.csv',low_memory=False)
firm_2008 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2008.csv',low_memory=False)
firm_2009 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2009.csv',low_memory=False)
firm_2010 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2010.csv',low_memory=False)
firm_2011 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2011.csv',low_memory=False)
firm_2012 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2012.csv',low_memory=False)
firm_2013 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2013.csv',low_memory=False)

In [4]:
firm_2007['鄉鎮碼'] = firm_2007['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2007['縣碼'] = firm_2007['行政区划代码'].astype(str).map(lambda x:x[0:6])

firm_2008['鄉鎮碼'] = firm_2008['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2008['縣碼'] = firm_2008['行政区划代码'].astype(str).map(lambda x:x[0:6])

firm_2009['鄉鎮碼'] = firm_2009['行政区别'].astype(str).map(lambda x:x[0:9])
firm_2009['縣碼'] = firm_2009['行政区别'].astype(str).map(lambda x:x[0:6])

firm_2010['鄉鎮碼'] = firm_2010['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2010['縣碼'] = firm_2010['行政区划代码'].astype(str).map(lambda x:x[0:6])

firm_2011['鄉鎮碼'] = firm_2011['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2011['縣碼'] = firm_2011['行政区划代码'].astype(str).map(lambda x:x[0:6])

firm_2012['鄉鎮碼'] = firm_2012['_行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2012['縣碼'] = firm_2012['_行政区划代码'].astype(str).map(lambda x:x[0:6])

firm_2013['鄉鎮碼'] = firm_2013['_行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2013['縣碼'] = firm_2013['_行政区划代码'].astype(str).map(lambda x:x[0:6])

In [82]:
final_9 = pd.read_excel(r'E:\paper\matching完畢_全時間_9碼.xlsx',index_col=0)
final_6 = pd.read_excel(r'E:\paper\matching完畢_全時間_6碼.xlsx',index_col=0)
final_9.index = final_9.index.astype(str)
final_6.index = final_6.index.astype(str)
final_9['縣碼'] = final_9['縣碼'].astype(str)

# 每年變量

In [84]:
final_9.drop(['比例','平均薪資','capital labor ratio','R&D OECD'],axis=1,inplace=True)
final_6.drop(['比例','平均薪資','capital labor ratio','R&D OECD'],axis=1,inplace=True)

In [85]:
# capital labor ratio
final_9['capital labor ratio'] = final_9.apply(lambda x:x['固定資產']/x['員工數'],axis=1)
final_6['capital labor ratio'] = final_6.apply(lambda x:x['固定資產']/x['員工數'],axis=1)
# log員工
final_9['log員工數'] = final_9['員工數'].map(lambda x:np.log(x))
final_6['log員工數'] = final_6['員工數'].map(lambda x:np.log(x))
# 人口&人口密度
pop_den = pd.read_excel(r'C:\Users\User\Dropbox\paper\科技園區\China\人口密度對照.xlsx',index_col=0)
pop_den.index = pop_den.index.astype(str)
final_9 = pd.merge(final_9,pop_den[['2010年人口','人口密度2010']],left_on='縣碼',right_index=True,how='left')
final_6 = pd.merge(final_6,pop_den[['2010年人口','人口密度2010']],left_index=True,right_index=True,how='left')
final_9['2010年人口'] = final_9['2010年人口'].map(lambda x:np.log(x))
final_6['2010年人口'] = final_6['2010年人口'].map(lambda x:np.log(x))
final_9['人口密度2010'] = final_9['人口密度2010'].map(lambda x:np.log(x))
final_6['人口密度2010'] = final_6['人口密度2010'].map(lambda x:np.log(x))

## 平均薪資

In [88]:
salary_2007_9 = firm_2007.groupby('鄉鎮碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()
salary_2007_6 = firm_2007.groupby('縣碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()

salary_2008_9 = firm_2008.groupby('鄉鎮碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()
salary_2008_6 = firm_2008.groupby('縣碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()

salary_2011_9 = firm_2011.groupby('鄉鎮碼')[['应付职工薪酬','从业人数']].sum()
salary_2011_6 = firm_2011.groupby('縣碼')[['应付职工薪酬','从业人数']].sum()

salary_2012_9 = firm_2012.groupby('鄉鎮碼')[['应付职工薪酬_本年贷方累计发生额_','_从业人数']].sum()
salary_2012_6 = firm_2012.groupby('縣碼')[['应付职工薪酬_本年贷方累计发生额_','_从业人数']].sum()

salary_2013_9 = firm_2013.groupby('鄉鎮碼')[['应付职工薪酬','_从业人数']].sum()
salary_2013_6 = firm_2013.groupby('縣碼')[['应付职工薪酬','_从业人数']].sum()

In [89]:
salary_2007_9['salary'] = salary_2007_9.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)
salary_2007_6['salary'] = salary_2007_6.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)

salary_2008_9['salary'] = salary_2008_9.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)
salary_2008_6['salary'] = salary_2008_6.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)

salary_2011_9['salary'] = salary_2011_9.apply(lambda x:x['应付职工薪酬']/x['从业人数'],axis=1)
salary_2011_6['salary'] = salary_2011_6.apply(lambda x:x['应付职工薪酬']/x['从业人数'],axis=1)

salary_2012_9['salary'] = salary_2012_9.apply(lambda x:x['应付职工薪酬_本年贷方累计发生额_']/x['_从业人数'],axis=1)
salary_2012_6['salary'] = salary_2012_6.apply(lambda x:x['应付职工薪酬_本年贷方累计发生额_']/x['_从业人数'],axis=1)

salary_2013_9['salary'] = salary_2013_9.apply(lambda x:x['应付职工薪酬']/x['_从业人数'],axis=1)
salary_2013_6['salary'] = salary_2013_6.apply(lambda x:x['应付职工薪酬']/x['_从业人数'],axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in longlong_scalars
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in longlong_scalars
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
D:\Anaconda\

In [90]:
salary_2007_9['year'] = 2007
salary_2007_6['year'] = 2007

salary_2008_9['year'] = 2008
salary_2008_6['year'] = 2008

salary_2011_9['year'] = 2011
salary_2011_6['year'] = 2011

salary_2012_9['year'] = 2012
salary_2012_6['year'] = 2012

salary_2013_9['year'] = 2013
salary_2013_6['year'] = 2013

In [91]:
salary_2007_9['比對'] = salary_2007_9.index
salary_2007_6['比對'] = salary_2007_6.index

salary_2008_9['比對'] = salary_2008_9.index
salary_2008_6['比對'] = salary_2008_6.index

salary_2011_9['比對'] = salary_2011_9.index
salary_2011_6['比對'] = salary_2011_6.index

salary_2012_9['比對'] = salary_2012_9.index
salary_2012_6['比對'] = salary_2012_6.index

salary_2013_9['比對'] = salary_2013_9.index
salary_2013_6['比對'] = salary_2013_6.index

In [92]:
salary_9 = pd.concat([salary_2007_9[['比對','salary','year']],
                   salary_2008_9[['比對','salary','year']],
                   salary_2011_9[['比對','salary','year']],
                   salary_2012_9[['比對','salary','year']],
                   salary_2013_9[['比對','salary','year']]],
         ignore_index=True).sort_values(['比對','year'])

salary_6 = pd.concat([salary_2007_6[['比對','salary','year']],
                   salary_2008_6[['比對','salary','year']],
                   salary_2011_6[['比對','salary','year']],
                   salary_2012_6[['比對','salary','year']],
                   salary_2013_6[['比對','salary','year']]],
         ignore_index=True).sort_values(['比對','year'])

In [93]:
final_9 = final_9.reset_index()
final_9 = pd.merge(final_9,salary_9[['比對','salary','year']],left_on=['鄉鎮碼','year'],right_on=['比對','year'],how='left')
final_9 = final_9.set_index('鄉鎮碼')
final_6 = final_6.reset_index()
final_6.rename(columns={"index":"縣碼"},inplace=True)
final_6 = pd.merge(final_6,salary_6[['比對','salary','year']],left_on=['縣碼','year'],right_on=['比對','year'],how='left')
final_6 = final_6.set_index('縣碼')

In [94]:
final_9 = final_9.reset_index()
final_9.drop('比對',axis=1,inplace=True)
final_6 = final_6.reset_index()
final_6.drop('比對',axis=1,inplace=True)
for i in list(final_9[final_9['year']==2009].index):
    final_9.loc[i,'salary'] = final_9.loc[i-1,'salary'] + ((final_9.loc[i+2,'salary'] - final_9.loc[i-1,'salary'])/3)
for i in list(final_9[final_9['year']==2010].index):
    final_9.loc[i,'salary'] = final_9.loc[i-2,'salary'] + (2*(final_9.loc[i+1,'salary'] - final_9.loc[i-2,'salary'])/3)
for i in list(final_6[final_6['year']==2009].index):
    final_6.loc[i,'salary'] = final_6.loc[i-1,'salary'] + ((final_6.loc[i+2,'salary'] - final_6.loc[i-1,'salary'])/3)
for i in list(final_6[final_6['year']==2010].index):
    final_6.loc[i,'salary'] = final_6.loc[i-2,'salary'] + (2*(final_6.loc[i+1,'salary'] - final_6.loc[i-2,'salary'])/3)
final_9 = final_9.set_index('鄉鎮碼')
final_6 = final_6.set_index('縣碼')

## 產業 & R&D

In [47]:
aircraft = [3761,3762,3769]
phar = [2710,2720,2730,2740,2750,2760,2770]
commu = [4011,4012,4019,4013,4014,4031,4032,4039,4071,4072,4051,4052,4053,4059,4061,4062]
office = [4041,4042,4043,4154,4155,2665,6211,6212]
precise = [3681,3682,3683,3684,3685,3686,3689,4111,4112,4113,4114,4115,4119,4121,4122,4123,4124,4125,4126,4127,4128,4129,4141,4190]

elec = ['39','40','41']
vehicle = ['37']
chem_phar = ['26','27']
equip = ['35','36']

rubber = ['28','29','30']
fuel = ['25']
nonmetal = ['31']
metal = ['32','33','34']

manu = ['42','43']
wood = ['20','21','22','23','24']
food = ['13','14','15','16']
textile = ['17','18','19']

In [48]:
def rnd(a,b):
    if a in aircraft:
        return 13.3
    elif a in phar:
        return 10.5
    elif a in commu:
        return 9.2
    elif a in office:
        return 8
    elif a in precise:
        return 7.7
    elif b in elec:
        return 3.9
    elif b in vehicle:
        return 3.5
    elif b in chem_phar:
        return 3.1
    elif b in equip:
        return 2.1
    elif b in rubber:
        return 0.9
    elif b in fuel:
        return 0.9
    elif b in nonmetal:
        return 0.9
    elif b in metal:
        return 0.6
    elif b in manu:
        return 0.5
    elif b in wood:
        return 0.3
    elif b in food:
        return 0.3
    elif b in textile:
        return 0.3
    else:
        return 0.3

In [49]:
high = aircraft+phar+commu+office+precise
mid_high = elec+vehicle+chem_phar+equip
mid_low = rubber+fuel+nonmetal+metal

In [50]:
firm_2007['產業二位碼'] = firm_2007['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2007['R&D OECD'] = firm_2007.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2008['產業二位碼'] = firm_2008['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2008['R&D OECD'] = firm_2008.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2009['產業二位碼'] = firm_2009['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2009['R&D OECD'] = firm_2009.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2010['產業二位碼'] = firm_2010['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2010['R&D OECD'] = firm_2010.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2011['產業二位碼'] = firm_2011['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2011['R&D OECD'] = firm_2011.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2012['產業二位碼'] = firm_2012['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2012['R&D OECD'] = firm_2012.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2013['產業二位碼'] = firm_2013['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2013['R&D OECD'] = firm_2013.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

In [51]:
firm_2007['high'] = firm_2007['行业代码'].map(lambda x:1 if x in high else 0)
firm_2007['mid_high'] = firm_2007.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2007['mid_low'] = firm_2007.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2008['high'] = firm_2008['行业代码'].map(lambda x:1 if x in high else 0)
firm_2008['mid_high'] = firm_2008.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2008['mid_low'] = firm_2008.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2009['high'] = firm_2009['行业代码'].map(lambda x:1 if x in high else 0)
firm_2009['mid_high'] = firm_2009.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2009['mid_low'] = firm_2009.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2010['high'] = firm_2010['行业代码'].map(lambda x:1 if x in high else 0)
firm_2010['mid_high'] = firm_2010.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2010['mid_low'] = firm_2010.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2011['high'] = firm_2011['行业代码'].map(lambda x:1 if x in high else 0)
firm_2011['mid_high'] = firm_2011.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2011['mid_low'] = firm_2011.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2012['high'] = firm_2012['行业代码'].map(lambda x:1 if x in high else 0)
firm_2012['mid_high'] = firm_2012.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2012['mid_low'] = firm_2012.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2013['high'] = firm_2013['行业代码'].map(lambda x:1 if x in high else 0)
firm_2013['mid_high'] = firm_2013.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2013['mid_low'] = firm_2013.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

### R&D

In [95]:
tech_per_9_2007 = firm_2007.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2008 = firm_2008.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2009 = firm_2009.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2010 = firm_2010.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2011 = firm_2011.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2012 = firm_2012.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2013 = firm_2013.groupby('鄉鎮碼')[['R&D OECD','high']].mean()

tech_per_6_2007 = firm_2007.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2008 = firm_2008.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2009 = firm_2009.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2010 = firm_2010.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2011 = firm_2011.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2012 = firm_2012.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2013 = firm_2013.groupby('縣碼')[['R&D OECD','high']].mean()

In [96]:
tech_per_9_2007['year'] = 2007
tech_per_6_2007['year'] = 2007

tech_per_9_2008['year'] = 2008
tech_per_6_2008['year'] = 2008

tech_per_9_2009['year'] = 2009
tech_per_6_2009['year'] = 2009

tech_per_9_2010['year'] = 2010
tech_per_6_2010['year'] = 2010

tech_per_9_2011['year'] = 2011
tech_per_6_2011['year'] = 2011

tech_per_9_2012['year'] = 2012
tech_per_6_2012['year'] = 2012

tech_per_9_2013['year'] = 2013
tech_per_6_2013['year'] = 2013

In [97]:
tech_per_9_2007['比對'] = tech_per_9_2007.index
tech_per_9_2008['比對'] = tech_per_9_2008.index
tech_per_9_2009['比對'] = tech_per_9_2009.index
tech_per_9_2010['比對'] = tech_per_9_2010.index
tech_per_9_2011['比對'] = tech_per_9_2011.index
tech_per_9_2012['比對'] = tech_per_9_2012.index
tech_per_9_2013['比對'] = tech_per_9_2013.index

tech_per_6_2007['比對'] = tech_per_6_2007.index
tech_per_6_2008['比對'] = tech_per_6_2008.index
tech_per_6_2009['比對'] = tech_per_6_2009.index
tech_per_6_2010['比對'] = tech_per_6_2010.index
tech_per_6_2011['比對'] = tech_per_6_2011.index
tech_per_6_2012['比對'] = tech_per_6_2012.index
tech_per_6_2013['比對'] = tech_per_6_2013.index

In [98]:
tech_per_9 = pd.concat([tech_per_9_2007[['比對','R&D OECD','year']],
                       tech_per_9_2008[['比對','R&D OECD','year']],
                       tech_per_9_2009[['比對','R&D OECD','year']],
                       tech_per_9_2010[['比對','R&D OECD','year']],
                       tech_per_9_2011[['比對','R&D OECD','year']],
                       tech_per_9_2012[['比對','R&D OECD','year']],
                       tech_per_9_2013[['比對','R&D OECD','year']]],
                    ignore_index=True).sort_values(['比對','year'])

tech_per_6 = pd.concat([tech_per_6_2007[['比對','R&D OECD','year']],
                       tech_per_6_2008[['比對','R&D OECD','year']],
                       tech_per_6_2009[['比對','R&D OECD','year']],
                       tech_per_6_2010[['比對','R&D OECD','year']],
                       tech_per_6_2011[['比對','R&D OECD','year']],
                       tech_per_6_2012[['比對','R&D OECD','year']],
                       tech_per_6_2013[['比對','R&D OECD','year']]],
                    ignore_index=True).sort_values(['比對','year'])

In [99]:
final_9 = final_9.reset_index()
final_9 = pd.merge(final_9,tech_per_9[['比對','R&D OECD','year']],left_on=['鄉鎮碼','year'],right_on=['比對','year'],how='left')
final_9 = final_9.set_index('鄉鎮碼')
final_9.drop('比對',axis=1,inplace=True)
final_6 = final_6.reset_index()
final_6.rename(columns={"index":"縣碼"},inplace=True)
final_6 = pd.merge(final_6,tech_per_6[['比對','R&D OECD','year']],left_on=['縣碼','year'],right_on=['比對','year'],how='left')
final_6 = final_6.set_index('縣碼')
final_6.drop('比對',axis=1,inplace=True)

### 產業比例

In [104]:
ind_per_9_2007 = firm_2007.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2008 = firm_2008.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2009 = firm_2009.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2010 = firm_2010.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2011 = firm_2011.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2012 = firm_2012.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2013 = firm_2013.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()

ind_per_6_2007 = firm_2007.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2008 = firm_2008.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2009 = firm_2009.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2010 = firm_2010.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2011 = firm_2011.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2012 = firm_2012.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2013 = firm_2013.groupby('縣碼')[['high','mid_high','mid_low']].sum()

In [105]:
ind_per_9_2007 = pd.merge(ind_per_9_2007,firm_2007.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2008 = pd.merge(ind_per_9_2008,firm_2008.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2009 = pd.merge(ind_per_9_2009,firm_2009.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2010 = pd.merge(ind_per_9_2010,firm_2010.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2011 = pd.merge(ind_per_9_2011,firm_2011.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2012 = pd.merge(ind_per_9_2012,firm_2012.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2013 = pd.merge(ind_per_9_2013,firm_2013.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')

ind_per_6_2007 = pd.merge(ind_per_6_2007,firm_2007.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2008 = pd.merge(ind_per_6_2008,firm_2008.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2009 = pd.merge(ind_per_6_2009,firm_2009.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2010 = pd.merge(ind_per_6_2010,firm_2010.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2011 = pd.merge(ind_per_6_2011,firm_2011.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2012 = pd.merge(ind_per_6_2012,firm_2012.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2013 = pd.merge(ind_per_6_2013,firm_2013.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')

In [106]:
ind_per_9_2007['year'] = 2007
ind_per_6_2007['year'] = 2007

ind_per_9_2008['year'] = 2008
ind_per_6_2008['year'] = 2008

ind_per_9_2009['year'] = 2009
ind_per_6_2009['year'] = 2009

ind_per_9_2010['year'] = 2010
ind_per_6_2010['year'] = 2010

ind_per_9_2011['year'] = 2011
ind_per_6_2011['year'] = 2011

ind_per_9_2012['year'] = 2012
ind_per_6_2012['year'] = 2012

ind_per_9_2013['year'] = 2013
ind_per_6_2013['year'] = 2013

In [107]:
ind_per_9_2007['比對'] = ind_per_9_2007.index
ind_per_9_2008['比對'] = ind_per_9_2008.index
ind_per_9_2009['比對'] = ind_per_9_2009.index
ind_per_9_2010['比對'] = ind_per_9_2010.index
ind_per_9_2011['比對'] = ind_per_9_2011.index
ind_per_9_2012['比對'] = ind_per_9_2012.index
ind_per_9_2013['比對'] = ind_per_9_2013.index

ind_per_6_2007['比對'] = ind_per_6_2007.index
ind_per_6_2008['比對'] = ind_per_6_2008.index
ind_per_6_2009['比對'] = ind_per_6_2009.index
ind_per_6_2010['比對'] = ind_per_6_2010.index
ind_per_6_2011['比對'] = ind_per_6_2011.index
ind_per_6_2012['比對'] = ind_per_6_2012.index
ind_per_6_2013['比對'] = ind_per_6_2013.index

In [108]:
ind_per_9 = pd.concat([ind_per_9_2007[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2008[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2009[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2010[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2011[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2012[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2013[['比對','high','mid_high','mid_low','行业代码','year']]],
                    ignore_index=True).sort_values(['比對','year'])

ind_per_6 = pd.concat([ind_per_6_2007[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2008[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2009[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2010[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2011[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2012[['比對','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2013[['比對','high','mid_high','mid_low','行业代码','year']]],
                    ignore_index=True).sort_values(['比對','year'])

In [109]:
ind_per_9['high_per'] = ind_per_9.apply(lambda x:x['high']/x['行业代码'],axis=1)
ind_per_9['mid_high_per'] = ind_per_9.apply(lambda x:x['mid_high']/x['行业代码'],axis=1)
ind_per_9['mid_low_per'] = ind_per_9.apply(lambda x:x['mid_low']/x['行业代码'],axis=1)
ind_per_6['high_per'] = ind_per_6.apply(lambda x:x['high']/x['行业代码'],axis=1)
ind_per_6['mid_high_per'] = ind_per_6.apply(lambda x:x['mid_high']/x['行业代码'],axis=1)
ind_per_6['mid_low_per'] = ind_per_6.apply(lambda x:x['mid_low']/x['行业代码'],axis=1)

In [112]:
final_9 = final_9.reset_index()
final_9 = pd.merge(final_9,ind_per_9[['比對','high_per','mid_high_per','mid_low_per','year']],left_on=['鄉鎮碼','year'],right_on=['比對','year'],how='left')
final_9 = final_9.set_index('鄉鎮碼')
final_9.drop('比對',axis=1,inplace=True)
final_6 = final_6.reset_index()
final_6.rename(columns={"index":"縣碼"},inplace=True)
final_6 = pd.merge(final_6,ind_per_6[['比對','high_per','mid_high_per','mid_low_per','year']],left_on=['縣碼','year'],right_on=['比對','year'],how='left')
final_6 = final_6.set_index('縣碼')
final_6.drop('比對',axis=1,inplace=True)

In [116]:
final_6

,TFP,year,市碼,固定資產,員工數,工業產值,舊園區,2010園區,2010後園區,時間點,after2010,treated,patent stock,cite stock,claim stock,capital labor ratio,log員工數,2010年人口,人口密度2010,salary,R&D OECD,high_per,mid_high_per,mid_low_per
縣碼,,,,,,,,,,,,,,,,,,,,,,,,
130203,4.104296,2007,1302,62158218.0,98488,78686884.0,0,1,0,-3,0,0,158.479270,1.308141,3.000092,631.124787,11.497690,13.519129,8.791742,26.659248,1.835577,0.076923,0.288462,0.451923
130203,4.315771,2008,1302,64062548.0,106664,103220920.0,0,1,0,-2,0,0,178.707379,1.428124,3.128771,600.601403,11.577439,13.519129,8.791742,38.913504,2.023810,0.076190,0.390476,0.380952
130203,4.088706,2009,1302,77659117.0,102456,86716561.0,0,1,0,-1,0,0,200.901272,1.333659,3.231758,757.975297,11.537189,13.519129,8.791742,49.606063,2.119091,0.090909,0.409091,0.345455
130203,4.434215,2010,1302,164203828.0,328637,332612196.0,0,1,0,0,0,0,231.766082,1.362338,3.205278,499.651068,12.702709,13.519129,8.791742,60.298622,1.809659,0.068182,0.295455,0.437500
130203,4.533840,2011,1302,89538590.0,92420,134674498.0,0,1,0,1,1,1,295.001169,1.418239,3.181152,968.822657,11.434099,13.519129,8.791742,70.991182,2.353012,0.108434,0.481928,0.253012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652301,4.232437,2009,6523,5725231.0,17285,12129496.0,0,1,0,-1,0,0,103.881816,1.578803,5.291125,331.225398,9.757594,12.962788,3.928589,33.535893,0.759615,0.000000,0.125000,0.278846
652301,3.827586,2010,6523,7126156.0,23796,10699015.0,0,1,0,0,0,0,123.299543,1.446945,5.581561,299.468650,10.077273,12.962788,3.928589,41.910679,0.901042,0.010417,0.145833,0.229167
652301,4.451580,2011,6523,5922601.0,24609,18921610.0,0,1,0,1,1,1,166.804612,1.280819,5.377389,240.668089,10.110868,12.962788,3.928589,50.285465,0.797938,0.000000,0.134021,0.257732


# 專利指標建構&合併

In [118]:
patent = pd.read_excel(r'E:\paper\專利資料(授權+新型)_統計數量.xlsx',index_col=0)
patent['比對'] = patent['比對'].astype(str)

In [161]:
year_list = []
for i in np.arange(1985,2016):
    k = 0
    while(k<113):
        year_list.append(i)
        k = k+1

In [162]:
patent_df = pd.DataFrame({'縣碼':list(final_6.index.unique())*31,'year':year_list})

In [163]:
patent_df = pd.merge(patent_df,patent[['比對','year','總專利數','總引用數']],left_on=['縣碼','year'],right_on=['比對','year'],how='left')

In [164]:
patent_df.drop('比對',axis=1,inplace=True)
patent_df.fillna(0,inplace=True)
patent_df = patent_df.sort_values(['縣碼','year'])
patent_df = patent_df.reset_index(drop=True)

In [165]:
for i in list(patent_df['縣碼'].unique()):
    for k in list(patent_df[patent_df['縣碼']==i].index):
        if patent_df.loc[k,'year']==1985:
            patent_df.loc[k,'patent stock'] = patent_df.loc[k,'總專利數']
            patent_df.loc[k,'cite'] = patent_df.loc[k,'總引用數']
        else:
            patent_df.loc[k,'patent stock'] = patent_df.loc[k-1,'patent stock']*0.85 + patent_df.loc[k,'總專利數']
            patent_df.loc[k,'cite'] = patent_df.loc[k-1,'cite']*0.85 + patent_df.loc[k,'總引用數']

In [166]:
patent_df['cite stock'] = patent_df.apply(lambda x:(x['cite']/x['patent stock']) if x['patent stock']!=0 else 0,axis=1)

In [167]:
patent_df['log patent stock'] = patent_df['patent stock'].map(lambda x:np.log(1+x))
patent_df['log cite stock'] = patent_df['cite stock'].map(lambda x:np.log(1+x))

In [169]:
for i in list(patent_df['縣碼'].unique()):
    for k in list(patent_df[patent_df['縣碼']==i].index):
        if patent_df.loc[k,'year']==1985:
            patent_df.loc[k,'patent stock growth'] = 0
        else:
            if (0.5*patent_df.loc[k,'patent stock'] + 0.5*patent_df.loc[k-1,'patent stock'])!=0:
                patent_df.loc[k,'patent stock growth'] = (patent_df.loc[k,'patent stock'] - patent_df.loc[k-1,'patent stock'])/(0.5*patent_df.loc[k,'patent stock'] + 0.5*patent_df.loc[k-1,'patent stock'])
            else:
                patent_df.loc[k,'patent stock growth'] = 0

for i in list(patent_df['縣碼'].unique()):
    for k in list(patent_df[patent_df['縣碼']==i].index):
        if patent_df.loc[k,'year']==1985:
            patent_df.loc[k,'cite stock growth'] = 0
        else:
            if (0.5*patent_df.loc[k,'cite stock'] + 0.5*patent_df.loc[k-1,'cite stock'])!=0:
                patent_df.loc[k,'cite stock growth'] = (patent_df.loc[k,'cite stock'] - patent_df.loc[k-1,'cite stock'])/(0.5*patent_df.loc[k,'cite stock'] + 0.5*patent_df.loc[k-1,'cite stock'])
            else:
                patent_df.loc[k,'cite stock growth'] = 0

In [170]:
patent_df

,縣碼,year,總專利數,總引用數,patent stock,cite,cite stock,log patent stock,log cite stock,patent stock growth,cite stock growth
0,130203,1985,2.0,0.0,2.000000,0.000000,0.000000,1.098612,0.000000,0.000000,0.000000
1,130203,1986,3.0,0.0,4.700000,0.000000,0.000000,1.740466,0.000000,0.805970,0.000000
2,130203,1987,4.0,1.0,7.995000,1.000000,0.125078,2.196669,0.117853,0.519102,2.000000
3,130203,1988,7.0,3.0,13.795750,3.850000,0.279071,2.694340,0.246134,0.532405,0.762061
4,130203,1989,5.0,30.0,16.726388,33.272500,1.989222,2.875054,1.095013,0.192034,1.507874
...,...,...,...,...,...,...,...,...,...,...,...
3498,652301,2011,62.0,62.0,166.804612,213.646468,1.280819,5.122800,0.824534,0.299927,-0.121804
3499,652301,2012,105.0,68.0,246.783920,249.599498,1.011409,5.512557,0.698836,0.386758,-0.235064
3500,652301,2013,103.0,76.0,312.766332,288.159573,0.921325,5.748649,0.653015,0.235841,-0.093219
3501,652301,2014,127.0,33.0,392.851382,277.935637,0.707483,5.975974,0.535020,0.226993,-0.262575


In [171]:
patent_df.to_excel(r'E:\paper\專利資料_matching對象.xlsx')

In [173]:
final_6.drop(['patent stock','cite stock','claim stock','log patent stock','log cite stock'],axis=1,inplace=True)

In [176]:
final_6 = final_6.reset_index()

In [179]:
final_6 = pd.merge(final_6,patent_df[['縣碼','year','patent stock','cite stock','log patent stock','log cite stock','patent stock growth','cite stock growth']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')
final_6 = final_6.set_index('縣碼')

In [182]:
final_9[final_9['year']!=2010].to_excel(r'E:\paper\新迴歸_9碼.xlsx')
final_6[final_6['year']!=2010].to_excel(r'E:\paper\新迴歸_6碼.xlsx')

# 變數lag一期

In [191]:
firm_2006 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2006.csv',low_memory=False)
firm_2006['鄉鎮碼'] = firm_2006['行政区码'].astype(str).map(lambda x:x[0:9])
firm_2006['縣碼'] = firm_2006['行政区码'].astype(str).map(lambda x:x[0:6])

In [233]:
final_9 = pd.read_excel(r'E:\paper\matching完畢_全時間_9碼.xlsx',index_col=0)
final_6 = pd.read_excel(r'E:\paper\matching完畢_全時間_6碼.xlsx',index_col=0)
final_9.index = final_9.index.astype(str)
final_6.index = final_6.index.astype(str)
final_9['縣碼'] = final_9['縣碼'].astype(str)
final_9.drop(['比例','平均薪資','capital labor ratio','R&D OECD'],axis=1,inplace=True)
final_6.drop(['比例','平均薪資','capital labor ratio','R&D OECD'],axis=1,inplace=True)

In [234]:
# capital labor ratio
final_9 = final_9.reset_index()
final_6 = final_6.reset_index()
final_9['capital labor ratio'] = final_9.apply(lambda x:x['固定資產']/x['員工數'],axis=1)
final_6['capital labor ratio'] = final_6.apply(lambda x:x['固定資產']/x['員工數'],axis=1)
final_9['capital labor ratio'] = final_9.groupby('鄉鎮碼')['capital labor ratio'].shift(1)
final_6['capital labor ratio'] = final_6.groupby('縣碼')['capital labor ratio'].shift(1)
#final_6.loc[final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index,'capital labor ratio'] = float(0.5*final_6.loc[(final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index)-1,'capital labor ratio']) + float(0.5*final_6.loc[(final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index)+1,'capital labor ratio'])

In [235]:
clratio_9 = firm_2006.groupby('鄉鎮碼')[['固定资产合','平均人数']].sum()
clratio_9['capital labor ratio'] = clratio_9.apply(lambda x:x['固定资产合']/x['平均人数'],axis=1)
clratio_6 = firm_2006.groupby('縣碼')[['固定资产合','平均人数']].sum()
clratio_6['capital labor ratio'] = clratio_6.apply(lambda x:x['固定资产合']/x['平均人数'],axis=1)
clratio_9['year'] = 2007
clratio_6['year'] = 2007
clratio_9 = clratio_9.reset_index()
clratio_6 = clratio_6.reset_index()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in longlong_scalars
  after removing the cwd from sys.path.


In [236]:
final_9 = pd.merge(final_9,clratio_9[['鄉鎮碼','year','capital labor ratio']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,clratio_6[['縣碼','year','capital labor ratio']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')
for i in final_9[final_9['year']==2007].index:
    final_9.loc[i,'capital labor ratio_x'] = final_9.loc[i,'capital labor ratio_y']
for i in final_6[final_6['year']==2007].index:
    final_6.loc[i,'capital labor ratio_x'] = final_6.loc[i,'capital labor ratio_y']
final_9.drop('capital labor ratio_y',axis=1,inplace=True)
final_9 = final_9.rename(columns={'capital labor ratio_x':'capital labor ratio'})
final_6.drop('capital labor ratio_y',axis=1,inplace=True)
final_6 = final_6.rename(columns={'capital labor ratio_x':'capital labor ratio'})

In [240]:
# log員工數
#final_6.loc[final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index,'員工數'] = float(0.5*final_6.loc[(final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index)-1,'員工數']) + float(0.5*final_6.loc[(final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index)+1,'員工數'])
final_9['log員工數'] = final_9['員工數'].map(lambda x:np.log(x))
final_6['log員工數'] = final_6['員工數'].map(lambda x:np.log(x))
final_9['log員工數'] = final_9.groupby('鄉鎮碼')['log員工數'].shift(1)
final_6['log員工數'] = final_6.groupby('縣碼')['log員工數'].shift(1)

In [242]:
clratio_9['log員工數'] = clratio_9['平均人数'].map(lambda x:np.log(x))
clratio_6['log員工數'] = clratio_6['平均人数'].map(lambda x:np.log(x))
final_9 = pd.merge(final_9,clratio_9[['鄉鎮碼','year','log員工數']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,clratio_6[['縣碼','year','log員工數']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')
for i in final_9[final_9['year']==2007].index:
    final_9.loc[i,'log員工數_x'] = final_9.loc[i,'log員工數_y']
for i in final_6[final_6['year']==2007].index:
    final_6.loc[i,'log員工數_x'] = final_6.loc[i,'log員工數_y']
final_9.drop('log員工數_y',axis=1,inplace=True)
final_9 = final_9.rename(columns={'log員工數_x':'log員工數'})
final_6.drop('log員工數_y',axis=1,inplace=True)
final_6 = final_6.rename(columns={'log員工數_x':'log員工數'})

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [248]:
# 人口&人口密度
pop_den = pd.read_excel(r'C:\Users\User\Dropbox\paper\科技園區\China\人口密度對照.xlsx',index_col=0)
pop_den.index = pop_den.index.astype(str)
final_9 = pd.merge(final_9,pop_den[['2010年人口','人口密度2010']],left_on='縣碼',right_index=True,how='left')
final_6 = pd.merge(final_6,pop_den[['2010年人口','人口密度2010']],left_on='縣碼',right_index=True,how='left')
final_9['2010年人口'] = final_9['2010年人口'].map(lambda x:np.log(x))
final_6['2010年人口'] = final_6['2010年人口'].map(lambda x:np.log(x))
final_9['人口密度2010'] = final_9['人口密度2010'].map(lambda x:np.log(x))
final_6['人口密度2010'] = final_6['人口密度2010'].map(lambda x:np.log(x))

In [250]:
# 平均薪資
salary_2006_9 = firm_2006.groupby('鄉鎮碼')[['应付工资','平均人数']].sum()
salary_2006_6 = firm_2006.groupby('縣碼')[['应付工资','平均人数']].sum()

salary_2007_9 = firm_2007.groupby('鄉鎮碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()
salary_2007_6 = firm_2007.groupby('縣碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()

salary_2008_9 = firm_2008.groupby('鄉鎮碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()
salary_2008_6 = firm_2008.groupby('縣碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()

salary_2011_9 = firm_2011.groupby('鄉鎮碼')[['应付职工薪酬','从业人数']].sum()
salary_2011_6 = firm_2011.groupby('縣碼')[['应付职工薪酬','从业人数']].sum()

salary_2012_9 = firm_2012.groupby('鄉鎮碼')[['应付职工薪酬_本年贷方累计发生额_','_从业人数']].sum()
salary_2012_6 = firm_2012.groupby('縣碼')[['应付职工薪酬_本年贷方累计发生额_','_从业人数']].sum()

In [251]:
salary_2006_9['salary'] = salary_2006_9.apply(lambda x:x['应付工资']/x['平均人数'],axis=1)
salary_2006_6['salary'] = salary_2006_6.apply(lambda x:x['应付工资']/x['平均人数'],axis=1)

salary_2007_9['salary'] = salary_2007_9.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)
salary_2007_6['salary'] = salary_2007_6.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)

salary_2008_9['salary'] = salary_2008_9.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)
salary_2008_6['salary'] = salary_2008_6.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)

salary_2011_9['salary'] = salary_2011_9.apply(lambda x:x['应付职工薪酬']/x['从业人数'],axis=1)
salary_2011_6['salary'] = salary_2011_6.apply(lambda x:x['应付职工薪酬']/x['从业人数'],axis=1)

salary_2012_9['salary'] = salary_2012_9.apply(lambda x:x['应付职工薪酬_本年贷方累计发生额_']/x['_从业人数'],axis=1)
salary_2012_6['salary'] = salary_2012_6.apply(lambda x:x['应付职工薪酬_本年贷方累计发生额_']/x['_从业人数'],axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in longlong_scalars
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in longlong_scalars
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in longlong_scalars
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in longlong_

In [252]:
salary_2006_9['year'] = 2007
salary_2006_6['year'] = 2007

salary_2007_9['year'] = 2008
salary_2007_6['year'] = 2008

salary_2008_9['year'] = 2009
salary_2008_6['year'] = 2009

salary_2011_9['year'] = 2012
salary_2011_6['year'] = 2012

salary_2012_9['year'] = 2013
salary_2012_6['year'] = 2013

In [253]:
salary_2006_9['鄉鎮碼'] = salary_2006_9.index
salary_2006_6['縣碼'] = salary_2006_6.index

salary_2007_9['鄉鎮碼'] = salary_2007_9.index
salary_2007_6['縣碼'] = salary_2007_6.index

salary_2008_9['鄉鎮碼'] = salary_2008_9.index
salary_2008_6['縣碼'] = salary_2008_6.index

salary_2011_9['鄉鎮碼'] = salary_2011_9.index
salary_2011_6['縣碼'] = salary_2011_6.index

salary_2012_9['鄉鎮碼'] = salary_2012_9.index
salary_2012_6['縣碼'] = salary_2012_6.index

In [255]:
salary_9 = pd.concat([salary_2006_9[['鄉鎮碼','salary','year']],
                   salary_2007_9[['鄉鎮碼','salary','year']],
                   salary_2008_9[['鄉鎮碼','salary','year']],
                   salary_2011_9[['鄉鎮碼','salary','year']],
                   salary_2012_9[['鄉鎮碼','salary','year']]],
         ignore_index=True).sort_values(['鄉鎮碼','year'])

salary_6 = pd.concat([salary_2006_6[['縣碼','salary','year']],
                   salary_2007_6[['縣碼','salary','year']],
                   salary_2008_6[['縣碼','salary','year']],
                   salary_2011_6[['縣碼','salary','year']],
                   salary_2012_6[['縣碼','salary','year']]],
         ignore_index=True).sort_values(['縣碼','year'])

In [256]:
final_9 = pd.merge(final_9,salary_9[['鄉鎮碼','salary','year']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,salary_6[['縣碼','salary','year']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')
for i in list(final_9[final_9['year']==2010].index):
    final_9.loc[i,'salary'] = final_9.loc[i-1,'salary'] + ((final_9.loc[i+2,'salary'] - final_9.loc[i-1,'salary'])/3)
for i in list(final_9[final_9['year']==2011].index):
    final_9.loc[i,'salary'] = final_9.loc[i-2,'salary'] + (2*(final_9.loc[i+1,'salary'] - final_9.loc[i-2,'salary'])/3)
for i in list(final_6[final_6['year']==2010].index):
    final_6.loc[i,'salary'] = final_6.loc[i-1,'salary'] + ((final_6.loc[i+2,'salary'] - final_6.loc[i-1,'salary'])/3)
for i in list(final_6[final_6['year']==2011].index):
    final_6.loc[i,'salary'] = final_6.loc[i-2,'salary'] + (2*(final_6.loc[i+1,'salary'] - final_6.loc[i-2,'salary'])/3)

In [259]:
# 產業
firm_2006['產業二位碼'] = firm_2006['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2006['R&D OECD'] = firm_2006.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2007['產業二位碼'] = firm_2007['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2007['R&D OECD'] = firm_2007.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2008['產業二位碼'] = firm_2008['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2008['R&D OECD'] = firm_2008.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2009['產業二位碼'] = firm_2009['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2009['R&D OECD'] = firm_2009.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2010['產業二位碼'] = firm_2010['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2010['R&D OECD'] = firm_2010.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2011['產業二位碼'] = firm_2011['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2011['R&D OECD'] = firm_2011.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2012['產業二位碼'] = firm_2012['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2012['R&D OECD'] = firm_2012.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

In [260]:
firm_2006['high'] = firm_2006['行业代码'].map(lambda x:1 if x in high else 0)
firm_2006['mid_high'] = firm_2006.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2006['mid_low'] = firm_2006.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2007['high'] = firm_2007['行业代码'].map(lambda x:1 if x in high else 0)
firm_2007['mid_high'] = firm_2007.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2007['mid_low'] = firm_2007.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2008['high'] = firm_2008['行业代码'].map(lambda x:1 if x in high else 0)
firm_2008['mid_high'] = firm_2008.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2008['mid_low'] = firm_2008.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2009['high'] = firm_2009['行业代码'].map(lambda x:1 if x in high else 0)
firm_2009['mid_high'] = firm_2009.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2009['mid_low'] = firm_2009.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2010['high'] = firm_2010['行业代码'].map(lambda x:1 if x in high else 0)
firm_2010['mid_high'] = firm_2010.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2010['mid_low'] = firm_2010.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2011['high'] = firm_2011['行业代码'].map(lambda x:1 if x in high else 0)
firm_2011['mid_high'] = firm_2011.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2011['mid_low'] = firm_2011.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2012['high'] = firm_2012['行业代码'].map(lambda x:1 if x in high else 0)
firm_2012['mid_high'] = firm_2012.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2012['mid_low'] = firm_2012.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

In [261]:
tech_per_9_2006 = firm_2006.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2007 = firm_2007.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2008 = firm_2008.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2009 = firm_2009.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2010 = firm_2010.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2011 = firm_2011.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2012 = firm_2012.groupby('鄉鎮碼')[['R&D OECD','high']].mean()

tech_per_6_2006 = firm_2006.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2007 = firm_2007.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2008 = firm_2008.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2009 = firm_2009.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2010 = firm_2010.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2011 = firm_2011.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2012 = firm_2012.groupby('縣碼')[['R&D OECD','high']].mean()

In [262]:
tech_per_9_2006['year'] = 2007
tech_per_6_2006['year'] = 2007

tech_per_9_2007['year'] = 2008
tech_per_6_2007['year'] = 2008

tech_per_9_2008['year'] = 2009
tech_per_6_2008['year'] = 2009

tech_per_9_2009['year'] = 2010
tech_per_6_2009['year'] = 2010

tech_per_9_2010['year'] = 2011
tech_per_6_2010['year'] = 2011

tech_per_9_2011['year'] = 2012
tech_per_6_2011['year'] = 2012

tech_per_9_2012['year'] = 2013
tech_per_6_2012['year'] = 2013

In [263]:
tech_per_9_2006['鄉鎮碼'] = tech_per_9_2006.index
tech_per_9_2007['鄉鎮碼'] = tech_per_9_2007.index
tech_per_9_2008['鄉鎮碼'] = tech_per_9_2008.index
tech_per_9_2009['鄉鎮碼'] = tech_per_9_2009.index
tech_per_9_2010['鄉鎮碼'] = tech_per_9_2010.index
tech_per_9_2011['鄉鎮碼'] = tech_per_9_2011.index
tech_per_9_2012['鄉鎮碼'] = tech_per_9_2012.index

tech_per_6_2006['縣碼'] = tech_per_6_2006.index
tech_per_6_2007['縣碼'] = tech_per_6_2007.index
tech_per_6_2008['縣碼'] = tech_per_6_2008.index
tech_per_6_2009['縣碼'] = tech_per_6_2009.index
tech_per_6_2010['縣碼'] = tech_per_6_2010.index
tech_per_6_2011['縣碼'] = tech_per_6_2011.index
tech_per_6_2012['縣碼'] = tech_per_6_2012.index

In [264]:
tech_per_9 = pd.concat([tech_per_9_2006[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2007[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2008[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2009[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2010[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2011[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2012[['鄉鎮碼','R&D OECD','year']]],
                    ignore_index=True).sort_values(['鄉鎮碼','year'])

tech_per_6 = pd.concat([tech_per_6_2006[['縣碼','R&D OECD','year']],
                       tech_per_6_2007[['縣碼','R&D OECD','year']],
                       tech_per_6_2008[['縣碼','R&D OECD','year']],
                       tech_per_6_2009[['縣碼','R&D OECD','year']],
                       tech_per_6_2010[['縣碼','R&D OECD','year']],
                       tech_per_6_2011[['縣碼','R&D OECD','year']],
                       tech_per_6_2012[['縣碼','R&D OECD','year']]],
                    ignore_index=True).sort_values(['縣碼','year'])

In [265]:
final_9 = pd.merge(final_9,tech_per_9[['鄉鎮碼','R&D OECD','year']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,tech_per_6[['縣碼','R&D OECD','year']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')

In [267]:
ind_per_9_2006 = firm_2006.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2007 = firm_2007.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2008 = firm_2008.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2009 = firm_2009.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2010 = firm_2010.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2011 = firm_2011.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2012 = firm_2012.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()

ind_per_6_2006 = firm_2006.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2007 = firm_2007.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2008 = firm_2008.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2009 = firm_2009.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2010 = firm_2010.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2011 = firm_2011.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2012 = firm_2012.groupby('縣碼')[['high','mid_high','mid_low']].sum()

In [268]:
ind_per_9_2006 = pd.merge(ind_per_9_2006,firm_2006.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2007 = pd.merge(ind_per_9_2007,firm_2007.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2008 = pd.merge(ind_per_9_2008,firm_2008.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2009 = pd.merge(ind_per_9_2009,firm_2009.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2010 = pd.merge(ind_per_9_2010,firm_2010.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2011 = pd.merge(ind_per_9_2011,firm_2011.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2012 = pd.merge(ind_per_9_2012,firm_2012.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')

ind_per_6_2006 = pd.merge(ind_per_6_2006,firm_2006.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2007 = pd.merge(ind_per_6_2007,firm_2007.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2008 = pd.merge(ind_per_6_2008,firm_2008.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2009 = pd.merge(ind_per_6_2009,firm_2009.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2010 = pd.merge(ind_per_6_2010,firm_2010.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2011 = pd.merge(ind_per_6_2011,firm_2011.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2012 = pd.merge(ind_per_6_2012,firm_2012.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')

In [269]:
ind_per_9_2006['year'] = 2007
ind_per_6_2006['year'] = 2007

ind_per_9_2007['year'] = 2008
ind_per_6_2007['year'] = 2008

ind_per_9_2008['year'] = 2009
ind_per_6_2008['year'] = 2009

ind_per_9_2009['year'] = 2010
ind_per_6_2009['year'] = 2010

ind_per_9_2010['year'] = 2011
ind_per_6_2010['year'] = 2011

ind_per_9_2011['year'] = 2012
ind_per_6_2011['year'] = 2012

ind_per_9_2012['year'] = 2013
ind_per_6_2012['year'] = 2013

In [270]:
ind_per_9_2006['鄉鎮碼'] = ind_per_9_2006.index
ind_per_9_2007['鄉鎮碼'] = ind_per_9_2007.index
ind_per_9_2008['鄉鎮碼'] = ind_per_9_2008.index
ind_per_9_2009['鄉鎮碼'] = ind_per_9_2009.index
ind_per_9_2010['鄉鎮碼'] = ind_per_9_2010.index
ind_per_9_2011['鄉鎮碼'] = ind_per_9_2011.index
ind_per_9_2012['鄉鎮碼'] = ind_per_9_2012.index

ind_per_6_2006['縣碼'] = ind_per_6_2006.index
ind_per_6_2007['縣碼'] = ind_per_6_2007.index
ind_per_6_2008['縣碼'] = ind_per_6_2008.index
ind_per_6_2009['縣碼'] = ind_per_6_2009.index
ind_per_6_2010['縣碼'] = ind_per_6_2010.index
ind_per_6_2011['縣碼'] = ind_per_6_2011.index
ind_per_6_2012['縣碼'] = ind_per_6_2012.index

In [271]:
ind_per_9 = pd.concat([ind_per_9_2006[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2007[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2008[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2009[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2010[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2011[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2012[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']]],
                    ignore_index=True).sort_values(['鄉鎮碼','year'])

ind_per_6 = pd.concat([ind_per_6_2006[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2007[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2008[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2009[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2010[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2011[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2012[['縣碼','high','mid_high','mid_low','行业代码','year']]],
                    ignore_index=True).sort_values(['縣碼','year'])

In [272]:
ind_per_9['high_per'] = ind_per_9.apply(lambda x:x['high']/x['行业代码'],axis=1)
ind_per_9['mid_high_per'] = ind_per_9.apply(lambda x:x['mid_high']/x['行业代码'],axis=1)
ind_per_9['mid_low_per'] = ind_per_9.apply(lambda x:x['mid_low']/x['行业代码'],axis=1)
ind_per_6['high_per'] = ind_per_6.apply(lambda x:x['high']/x['行业代码'],axis=1)
ind_per_6['mid_high_per'] = ind_per_6.apply(lambda x:x['mid_high']/x['行业代码'],axis=1)
ind_per_6['mid_low_per'] = ind_per_6.apply(lambda x:x['mid_low']/x['行业代码'],axis=1)

In [273]:
final_9 = pd.merge(final_9,ind_per_9[['鄉鎮碼','high_per','mid_high_per','mid_low_per','year']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,ind_per_6[['縣碼','high_per','mid_high_per','mid_low_per','year']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')

In [277]:
# 專利資料
patent_df = pd.read_excel(r'E:\paper\專利資料_matching對象.xlsx',index_col=0)
patent_df['縣碼'] = patent_df['縣碼'].astype(str)

In [279]:
final_6.drop(['patent stock','cite stock','claim stock'],axis=1,inplace=True)
final_9.drop(['patent stock','平均被引用','平均權利要求'],axis=1,inplace=True)
final_6 = pd.merge(final_6,patent_df[['縣碼','year','patent stock','cite stock','log patent stock','log cite stock','patent stock growth','cite stock growth']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')

In [282]:
final_9 = final_9.set_index('鄉鎮碼')
final_6 = final_6.set_index('縣碼')
final_9['year'] = final_9['year'].astype(str)
final_6['year'] = final_6['year'].astype(str)

In [287]:
final_9[final_9['year']!='2010'].to_excel(r'E:\paper\新迴歸_9碼_lag1.xlsx')
final_6[final_6['year']!='2010'].to_excel(r'E:\paper\新迴歸_6碼_lag1.xlsx')

# 變數lag兩期

In [288]:
firm_2005 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2005.csv',low_memory=False)
firm_2006 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2006.csv',low_memory=False)
firm_2007 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2007.csv',low_memory=False)
firm_2008 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2008.csv',low_memory=False)
firm_2009 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2009.csv',low_memory=False)
firm_2010 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2010.csv',low_memory=False)
firm_2011 = pd.read_csv(r'E:\paper\Firm csv\firm_data_2011.csv',low_memory=False)

In [289]:
firm_2005['鄉鎮碼'] = firm_2005['行政区代码'].astype(str).map(lambda x:x[0:9])
firm_2005['縣碼'] = firm_2005['行政区代码'].astype(str).map(lambda x:x[0:6])

firm_2006['鄉鎮碼'] = firm_2006['行政区码'].astype(str).map(lambda x:x[0:9])
firm_2006['縣碼'] = firm_2006['行政区码'].astype(str).map(lambda x:x[0:6])

firm_2007['鄉鎮碼'] = firm_2007['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2007['縣碼'] = firm_2007['行政区划代码'].astype(str).map(lambda x:x[0:6])

firm_2008['鄉鎮碼'] = firm_2008['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2008['縣碼'] = firm_2008['行政区划代码'].astype(str).map(lambda x:x[0:6])

firm_2009['鄉鎮碼'] = firm_2009['行政区别'].astype(str).map(lambda x:x[0:9])
firm_2009['縣碼'] = firm_2009['行政区别'].astype(str).map(lambda x:x[0:6])

firm_2010['鄉鎮碼'] = firm_2010['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2010['縣碼'] = firm_2010['行政区划代码'].astype(str).map(lambda x:x[0:6])

firm_2011['鄉鎮碼'] = firm_2011['行政区划代码'].astype(str).map(lambda x:x[0:9])
firm_2011['縣碼'] = firm_2011['行政区划代码'].astype(str).map(lambda x:x[0:6])

In [290]:
final_9 = pd.read_excel(r'E:\paper\matching完畢_全時間_9碼.xlsx',index_col=0)
final_6 = pd.read_excel(r'E:\paper\matching完畢_全時間_6碼.xlsx',index_col=0)
final_9.index = final_9.index.astype(str)
final_6.index = final_6.index.astype(str)
final_9['縣碼'] = final_9['縣碼'].astype(str)
final_9.drop(['比例','平均薪資','capital labor ratio','R&D OECD'],axis=1,inplace=True)
final_6.drop(['比例','平均薪資','capital labor ratio','R&D OECD'],axis=1,inplace=True)

In [291]:
# capital labor ratio
final_9 = final_9.reset_index()
final_6 = final_6.reset_index()
final_9['capital labor ratio'] = final_9.apply(lambda x:x['固定資產']/x['員工數'],axis=1)
final_6['capital labor ratio'] = final_6.apply(lambda x:x['固定資產']/x['員工數'],axis=1)
final_9['capital labor ratio'] = final_9.groupby('鄉鎮碼')['capital labor ratio'].shift(2)
final_6['capital labor ratio'] = final_6.groupby('縣碼')['capital labor ratio'].shift(2)

In [294]:
clratio_9_2006 = firm_2006.groupby('鄉鎮碼')[['固定资产合','平均人数']].sum()
clratio_9_2006['capital labor ratio'] = clratio_9_2006.apply(lambda x:x['固定资产合']/x['平均人数'],axis=1)
clratio_6_2006 = firm_2006.groupby('縣碼')[['固定资产合','平均人数']].sum()
clratio_6_2006['capital labor ratio'] = clratio_6_2006.apply(lambda x:x['固定资产合']/x['平均人数'],axis=1)
clratio_9_2006['year'] = 2008
clratio_6_2006['year'] = 2008
clratio_9_2006 = clratio_9_2006.reset_index()
clratio_6_2006 = clratio_6_2006.reset_index()

clratio_9_2005 = firm_2005.groupby('鄉鎮碼')[['固定资产合','从业人员均']].sum()
clratio_9_2005['capital labor ratio'] = clratio_9_2005.apply(lambda x:x['固定资产合']/x['从业人员均'],axis=1)
clratio_6_2005 = firm_2005.groupby('縣碼')[['固定资产合','从业人员均']].sum()
clratio_6_2005['capital labor ratio'] = clratio_6_2005.apply(lambda x:x['固定资产合']/x['从业人员均'],axis=1)
clratio_9_2005['year'] = 2007
clratio_6_2005['year'] = 2007
clratio_9_2005 = clratio_9_2005.reset_index()
clratio_6_2005 = clratio_6_2005.reset_index()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in longlong_scalars
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encou

In [295]:
clratio_9 = pd.concat([clratio_9_2005[['鄉鎮碼','capital labor ratio','year']],
                       clratio_9_2006[['鄉鎮碼','capital labor ratio','year']]],
                    ignore_index=True).sort_values(['鄉鎮碼','year'])

clratio_6 = pd.concat([clratio_6_2005[['縣碼','capital labor ratio','year']],
                       clratio_6_2006[['縣碼','capital labor ratio','year']]],
                    ignore_index=True).sort_values(['縣碼','year'])

In [297]:
final_9 = pd.merge(final_9,clratio_9[['鄉鎮碼','year','capital labor ratio']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,clratio_6[['縣碼','year','capital labor ratio']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')
for i in final_9[final_9['year']==2007].index:
    final_9.loc[i,'capital labor ratio_x'] = final_9.loc[i,'capital labor ratio_y']
for i in final_6[final_6['year']==2007].index:
    final_6.loc[i,'capital labor ratio_x'] = final_6.loc[i,'capital labor ratio_y']
for i in final_9[final_9['year']==2008].index:
    final_9.loc[i,'capital labor ratio_x'] = final_9.loc[i,'capital labor ratio_y']
for i in final_6[final_6['year']==2008].index:
    final_6.loc[i,'capital labor ratio_x'] = final_6.loc[i,'capital labor ratio_y']
final_9.drop('capital labor ratio_y',axis=1,inplace=True)
final_9 = final_9.rename(columns={'capital labor ratio_x':'capital labor ratio'})
final_6.drop('capital labor ratio_y',axis=1,inplace=True)
final_6 = final_6.rename(columns={'capital labor ratio_x':'capital labor ratio'})

In [303]:
# log員工數
#final_6.loc[final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index,'員工數'] = float(0.5*final_6.loc[(final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index)-1,'員工數']) + float(0.5*final_6.loc[(final_6[(final_6['縣碼']=='420502')&(final_6['year']==2011)].index)+1,'員工數'])
final_9['log員工數'] = final_9['員工數'].map(lambda x:np.log(x))
final_6['log員工數'] = final_6['員工數'].map(lambda x:np.log(x))
final_9['log員工數'] = final_9.groupby('鄉鎮碼')['log員工數'].shift(2)
final_6['log員工數'] = final_6.groupby('縣碼')['log員工數'].shift(2)

In [305]:
clratio_9_2006['log員工數'] = clratio_9_2006['平均人数'].map(lambda x:np.log(x))
clratio_6_2006['log員工數'] = clratio_6_2006['平均人数'].map(lambda x:np.log(x))
final_9 = pd.merge(final_9,clratio_9_2006[['鄉鎮碼','year','log員工數']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,clratio_6_2006[['縣碼','year','log員工數']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')
for i in final_9[final_9['year']==2008].index:
    final_9.loc[i,'log員工數_x'] = final_9.loc[i,'log員工數_y']
for i in final_6[final_6['year']==2008].index:
    final_6.loc[i,'log員工數_x'] = final_6.loc[i,'log員工數_y']
final_9.drop('log員工數_y',axis=1,inplace=True)
final_9 = final_9.rename(columns={'log員工數_x':'log員工數'})
final_6.drop('log員工數_y',axis=1,inplace=True)
final_6 = final_6.rename(columns={'log員工數_x':'log員工數'})

clratio_9_2005['log員工數'] = clratio_9_2005['从业人员均'].map(lambda x:np.log(x))
clratio_6_2005['log員工數'] = clratio_6_2005['从业人员均'].map(lambda x:np.log(x))
final_9 = pd.merge(final_9,clratio_9_2005[['鄉鎮碼','year','log員工數']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,clratio_6_2005[['縣碼','year','log員工數']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')
for i in final_9[final_9['year']==2007].index:
    final_9.loc[i,'log員工數_x'] = final_9.loc[i,'log員工數_y']
for i in final_6[final_6['year']==2007].index:
    final_6.loc[i,'log員工數_x'] = final_6.loc[i,'log員工數_y']
final_9.drop('log員工數_y',axis=1,inplace=True)
final_9 = final_9.rename(columns={'log員工數_x':'log員工數'})
final_6.drop('log員工數_y',axis=1,inplace=True)
final_6 = final_6.rename(columns={'log員工數_x':'log員工數'})

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


In [308]:
# 人口&人口密度
pop_den = pd.read_excel(r'C:\Users\User\Dropbox\paper\科技園區\China\人口密度對照.xlsx',index_col=0)
pop_den.index = pop_den.index.astype(str)
final_9 = pd.merge(final_9,pop_den[['2010年人口','人口密度2010']],left_on='縣碼',right_index=True,how='left')
final_6 = pd.merge(final_6,pop_den[['2010年人口','人口密度2010']],left_on='縣碼',right_index=True,how='left')
final_9['2010年人口'] = final_9['2010年人口'].map(lambda x:np.log(x))
final_6['2010年人口'] = final_6['2010年人口'].map(lambda x:np.log(x))
final_9['人口密度2010'] = final_9['人口密度2010'].map(lambda x:np.log(x))
final_6['人口密度2010'] = final_6['人口密度2010'].map(lambda x:np.log(x))

In [310]:
# 平均薪資
salary_2005_9 = firm_2005.groupby('鄉鎮碼')[['应付工资总','从业人员均']].sum()
salary_2005_6 = firm_2005.groupby('縣碼')[['应付工资总','从业人员均']].sum()

salary_2006_9 = firm_2006.groupby('鄉鎮碼')[['应付工资','平均人数']].sum()
salary_2006_6 = firm_2006.groupby('縣碼')[['应付工资','平均人数']].sum()

salary_2007_9 = firm_2007.groupby('鄉鎮碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()
salary_2007_6 = firm_2007.groupby('縣碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()

salary_2008_9 = firm_2008.groupby('鄉鎮碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()
salary_2008_6 = firm_2008.groupby('縣碼')[['本年应付工资总额(贷方累计发生额)','全部从业人员年平均人数']].sum()

salary_2011_9 = firm_2011.groupby('鄉鎮碼')[['应付职工薪酬','从业人数']].sum()
salary_2011_6 = firm_2011.groupby('縣碼')[['应付职工薪酬','从业人数']].sum()

In [311]:
salary_2005_9['salary'] = salary_2005_9.apply(lambda x:x['应付工资总']/x['从业人员均'],axis=1)
salary_2005_6['salary'] = salary_2005_6.apply(lambda x:x['应付工资总']/x['从业人员均'],axis=1)

salary_2006_9['salary'] = salary_2006_9.apply(lambda x:x['应付工资']/x['平均人数'],axis=1)
salary_2006_6['salary'] = salary_2006_6.apply(lambda x:x['应付工资']/x['平均人数'],axis=1)

salary_2007_9['salary'] = salary_2007_9.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)
salary_2007_6['salary'] = salary_2007_6.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)

salary_2008_9['salary'] = salary_2008_9.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)
salary_2008_6['salary'] = salary_2008_6.apply(lambda x:x['本年应付工资总额(贷方累计发生额)']/x['全部从业人员年平均人数'],axis=1)

salary_2011_9['salary'] = salary_2011_9.apply(lambda x:x['应付职工薪酬']/x['从业人数'],axis=1)
salary_2011_6['salary'] = salary_2011_6.apply(lambda x:x['应付职工薪酬']/x['从业人数'],axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in longlong_scalars
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in longlong_scalars
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in longlong_scalars
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in longlong_scalars
  """
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in longlong_scalars
  """
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  import sys
D:\Anaconda\lib\site-

In [312]:
salary_2005_9['year'] = 2007
salary_2005_6['year'] = 2007

salary_2006_9['year'] = 2008
salary_2006_6['year'] = 2008

salary_2007_9['year'] = 2009
salary_2007_6['year'] = 2009

salary_2008_9['year'] = 2010
salary_2008_6['year'] = 2010

salary_2011_9['year'] = 2013
salary_2011_6['year'] = 2013

In [313]:
salary_2005_9['鄉鎮碼'] = salary_2005_9.index
salary_2005_6['縣碼'] = salary_2005_6.index

salary_2006_9['鄉鎮碼'] = salary_2006_9.index
salary_2006_6['縣碼'] = salary_2006_6.index

salary_2007_9['鄉鎮碼'] = salary_2007_9.index
salary_2007_6['縣碼'] = salary_2007_6.index

salary_2008_9['鄉鎮碼'] = salary_2008_9.index
salary_2008_6['縣碼'] = salary_2008_6.index

salary_2011_9['鄉鎮碼'] = salary_2011_9.index
salary_2011_6['縣碼'] = salary_2011_6.index

In [314]:
salary_9 = pd.concat([salary_2005_9[['鄉鎮碼','salary','year']],
                   salary_2006_9[['鄉鎮碼','salary','year']],
                   salary_2007_9[['鄉鎮碼','salary','year']],
                   salary_2008_9[['鄉鎮碼','salary','year']],
                   salary_2011_9[['鄉鎮碼','salary','year']]],
         ignore_index=True).sort_values(['鄉鎮碼','year'])

salary_6 = pd.concat([salary_2005_6[['縣碼','salary','year']],
                   salary_2006_6[['縣碼','salary','year']],
                   salary_2007_6[['縣碼','salary','year']],
                   salary_2008_6[['縣碼','salary','year']],
                   salary_2011_6[['縣碼','salary','year']]],
         ignore_index=True).sort_values(['縣碼','year'])

In [315]:
final_9 = pd.merge(final_9,salary_9[['鄉鎮碼','salary','year']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,salary_6[['縣碼','salary','year']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')
for i in list(final_9[final_9['year']==2011].index):
    final_9.loc[i,'salary'] = final_9.loc[i-1,'salary'] + ((final_9.loc[i+2,'salary'] - final_9.loc[i-1,'salary'])/3)
for i in list(final_9[final_9['year']==2012].index):
    final_9.loc[i,'salary'] = final_9.loc[i-2,'salary'] + (2*(final_9.loc[i+1,'salary'] - final_9.loc[i-2,'salary'])/3)
for i in list(final_6[final_6['year']==2011].index):
    final_6.loc[i,'salary'] = final_6.loc[i-1,'salary'] + ((final_6.loc[i+2,'salary'] - final_6.loc[i-1,'salary'])/3)
for i in list(final_6[final_6['year']==2012].index):
    final_6.loc[i,'salary'] = final_6.loc[i-2,'salary'] + (2*(final_6.loc[i+1,'salary'] - final_6.loc[i-2,'salary'])/3)

In [317]:
# 產業
firm_2005['產業二位碼'] = firm_2005['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2005['R&D OECD'] = firm_2005.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2006['產業二位碼'] = firm_2006['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2006['R&D OECD'] = firm_2006.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2007['產業二位碼'] = firm_2007['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2007['R&D OECD'] = firm_2007.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2008['產業二位碼'] = firm_2008['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2008['R&D OECD'] = firm_2008.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2009['產業二位碼'] = firm_2009['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2009['R&D OECD'] = firm_2009.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2010['產業二位碼'] = firm_2010['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2010['R&D OECD'] = firm_2010.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

firm_2011['產業二位碼'] = firm_2011['行业代码'].astype(str).map(lambda x:x[:-2])
firm_2011['R&D OECD'] = firm_2011.apply(lambda x:rnd(x['行业代码'],x['產業二位碼']),axis=1)

In [318]:
firm_2005['high'] = firm_2005['行业代码'].map(lambda x:1 if x in high else 0)
firm_2005['mid_high'] = firm_2005.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2005['mid_low'] = firm_2005.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2006['high'] = firm_2006['行业代码'].map(lambda x:1 if x in high else 0)
firm_2006['mid_high'] = firm_2006.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2006['mid_low'] = firm_2006.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2007['high'] = firm_2007['行业代码'].map(lambda x:1 if x in high else 0)
firm_2007['mid_high'] = firm_2007.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2007['mid_low'] = firm_2007.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2008['high'] = firm_2008['行业代码'].map(lambda x:1 if x in high else 0)
firm_2008['mid_high'] = firm_2008.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2008['mid_low'] = firm_2008.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2009['high'] = firm_2009['行业代码'].map(lambda x:1 if x in high else 0)
firm_2009['mid_high'] = firm_2009.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2009['mid_low'] = firm_2009.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2010['high'] = firm_2010['行业代码'].map(lambda x:1 if x in high else 0)
firm_2010['mid_high'] = firm_2010.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2010['mid_low'] = firm_2010.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

firm_2011['high'] = firm_2011['行业代码'].map(lambda x:1 if x in high else 0)
firm_2011['mid_high'] = firm_2011.apply(lambda x:1 if ((x['產業二位碼'] in mid_high)&(x['high']==0)) else 0,axis=1)
firm_2011['mid_low'] = firm_2011.apply(lambda x:1 if ((x['產業二位碼'] in mid_low)&(x['high']==0)) else 0,axis=1)

In [319]:
tech_per_9_2005 = firm_2005.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2006 = firm_2006.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2007 = firm_2007.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2008 = firm_2008.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2009 = firm_2009.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2010 = firm_2010.groupby('鄉鎮碼')[['R&D OECD','high']].mean()
tech_per_9_2011 = firm_2011.groupby('鄉鎮碼')[['R&D OECD','high']].mean()

tech_per_6_2005 = firm_2005.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2006 = firm_2006.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2007 = firm_2007.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2008 = firm_2008.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2009 = firm_2009.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2010 = firm_2010.groupby('縣碼')[['R&D OECD','high']].mean()
tech_per_6_2011 = firm_2011.groupby('縣碼')[['R&D OECD','high']].mean()

In [320]:
tech_per_9_2005['year'] = 2007
tech_per_6_2005['year'] = 2007

tech_per_9_2006['year'] = 2008
tech_per_6_2006['year'] = 2008

tech_per_9_2007['year'] = 2009
tech_per_6_2007['year'] = 2009

tech_per_9_2008['year'] = 2010
tech_per_6_2008['year'] = 2010

tech_per_9_2009['year'] = 2011
tech_per_6_2009['year'] = 2011

tech_per_9_2010['year'] = 2012
tech_per_6_2010['year'] = 2012

tech_per_9_2011['year'] = 2013
tech_per_6_2011['year'] = 2013

In [321]:
tech_per_9_2005['鄉鎮碼'] = tech_per_9_2005.index
tech_per_9_2006['鄉鎮碼'] = tech_per_9_2006.index
tech_per_9_2007['鄉鎮碼'] = tech_per_9_2007.index
tech_per_9_2008['鄉鎮碼'] = tech_per_9_2008.index
tech_per_9_2009['鄉鎮碼'] = tech_per_9_2009.index
tech_per_9_2010['鄉鎮碼'] = tech_per_9_2010.index
tech_per_9_2011['鄉鎮碼'] = tech_per_9_2011.index

tech_per_6_2005['縣碼'] = tech_per_6_2005.index
tech_per_6_2006['縣碼'] = tech_per_6_2006.index
tech_per_6_2007['縣碼'] = tech_per_6_2007.index
tech_per_6_2008['縣碼'] = tech_per_6_2008.index
tech_per_6_2009['縣碼'] = tech_per_6_2009.index
tech_per_6_2010['縣碼'] = tech_per_6_2010.index
tech_per_6_2011['縣碼'] = tech_per_6_2011.index

In [322]:
tech_per_9 = pd.concat([tech_per_9_2005[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2006[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2007[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2008[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2009[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2010[['鄉鎮碼','R&D OECD','year']],
                       tech_per_9_2011[['鄉鎮碼','R&D OECD','year']]],
                    ignore_index=True).sort_values(['鄉鎮碼','year'])

tech_per_6 = pd.concat([tech_per_6_2005[['縣碼','R&D OECD','year']],
                       tech_per_6_2006[['縣碼','R&D OECD','year']],
                       tech_per_6_2007[['縣碼','R&D OECD','year']],
                       tech_per_6_2008[['縣碼','R&D OECD','year']],
                       tech_per_6_2009[['縣碼','R&D OECD','year']],
                       tech_per_6_2010[['縣碼','R&D OECD','year']],
                       tech_per_6_2011[['縣碼','R&D OECD','year']]],
                    ignore_index=True).sort_values(['縣碼','year'])

In [323]:
final_9 = pd.merge(final_9,tech_per_9[['鄉鎮碼','R&D OECD','year']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,tech_per_6[['縣碼','R&D OECD','year']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')

In [325]:
ind_per_9_2005 = firm_2005.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2006 = firm_2006.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2007 = firm_2007.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2008 = firm_2008.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2009 = firm_2009.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2010 = firm_2010.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()
ind_per_9_2011 = firm_2011.groupby('鄉鎮碼')[['high','mid_high','mid_low']].sum()

ind_per_6_2005 = firm_2005.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2006 = firm_2006.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2007 = firm_2007.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2008 = firm_2008.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2009 = firm_2009.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2010 = firm_2010.groupby('縣碼')[['high','mid_high','mid_low']].sum()
ind_per_6_2011 = firm_2011.groupby('縣碼')[['high','mid_high','mid_low']].sum()

In [326]:
ind_per_9_2005 = pd.merge(ind_per_9_2005,firm_2005.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2006 = pd.merge(ind_per_9_2006,firm_2006.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2007 = pd.merge(ind_per_9_2007,firm_2007.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2008 = pd.merge(ind_per_9_2008,firm_2008.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2009 = pd.merge(ind_per_9_2009,firm_2009.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2010 = pd.merge(ind_per_9_2010,firm_2010.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_9_2011 = pd.merge(ind_per_9_2011,firm_2011.groupby('鄉鎮碼')['行业代码'].count(),left_index=True,right_index=True,how='left')

ind_per_6_2005 = pd.merge(ind_per_6_2005,firm_2005.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2006 = pd.merge(ind_per_6_2006,firm_2006.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2007 = pd.merge(ind_per_6_2007,firm_2007.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2008 = pd.merge(ind_per_6_2008,firm_2008.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2009 = pd.merge(ind_per_6_2009,firm_2009.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2010 = pd.merge(ind_per_6_2010,firm_2010.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')
ind_per_6_2011 = pd.merge(ind_per_6_2011,firm_2011.groupby('縣碼')['行业代码'].count(),left_index=True,right_index=True,how='left')

In [327]:
ind_per_9_2005['year'] = 2007
ind_per_6_2005['year'] = 2007

ind_per_9_2006['year'] = 2008
ind_per_6_2006['year'] = 2008

ind_per_9_2007['year'] = 2009
ind_per_6_2007['year'] = 2009

ind_per_9_2008['year'] = 2010
ind_per_6_2008['year'] = 2010

ind_per_9_2009['year'] = 2011
ind_per_6_2009['year'] = 2011

ind_per_9_2010['year'] = 2012
ind_per_6_2010['year'] = 2012

ind_per_9_2011['year'] = 2013
ind_per_6_2011['year'] = 2013

In [328]:
ind_per_9_2005['鄉鎮碼'] = ind_per_9_2005.index
ind_per_9_2006['鄉鎮碼'] = ind_per_9_2006.index
ind_per_9_2007['鄉鎮碼'] = ind_per_9_2007.index
ind_per_9_2008['鄉鎮碼'] = ind_per_9_2008.index
ind_per_9_2009['鄉鎮碼'] = ind_per_9_2009.index
ind_per_9_2010['鄉鎮碼'] = ind_per_9_2010.index
ind_per_9_2011['鄉鎮碼'] = ind_per_9_2011.index

ind_per_6_2005['縣碼'] = ind_per_6_2005.index
ind_per_6_2006['縣碼'] = ind_per_6_2006.index
ind_per_6_2007['縣碼'] = ind_per_6_2007.index
ind_per_6_2008['縣碼'] = ind_per_6_2008.index
ind_per_6_2009['縣碼'] = ind_per_6_2009.index
ind_per_6_2010['縣碼'] = ind_per_6_2010.index
ind_per_6_2011['縣碼'] = ind_per_6_2011.index

In [329]:
ind_per_9 = pd.concat([ind_per_9_2005[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2006[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2007[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2008[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2009[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2010[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_9_2011[['鄉鎮碼','high','mid_high','mid_low','行业代码','year']]],
                    ignore_index=True).sort_values(['鄉鎮碼','year'])

ind_per_6 = pd.concat([ind_per_6_2005[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2006[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2007[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2008[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2009[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2010[['縣碼','high','mid_high','mid_low','行业代码','year']],
                       ind_per_6_2011[['縣碼','high','mid_high','mid_low','行业代码','year']]],
                    ignore_index=True).sort_values(['縣碼','year'])

In [330]:
ind_per_9['high_per'] = ind_per_9.apply(lambda x:x['high']/x['行业代码'],axis=1)
ind_per_9['mid_high_per'] = ind_per_9.apply(lambda x:x['mid_high']/x['行业代码'],axis=1)
ind_per_9['mid_low_per'] = ind_per_9.apply(lambda x:x['mid_low']/x['行业代码'],axis=1)
ind_per_6['high_per'] = ind_per_6.apply(lambda x:x['high']/x['行业代码'],axis=1)
ind_per_6['mid_high_per'] = ind_per_6.apply(lambda x:x['mid_high']/x['行业代码'],axis=1)
ind_per_6['mid_low_per'] = ind_per_6.apply(lambda x:x['mid_low']/x['行业代码'],axis=1)

In [331]:
final_9 = pd.merge(final_9,ind_per_9[['鄉鎮碼','high_per','mid_high_per','mid_low_per','year']],left_on=['鄉鎮碼','year'],right_on=['鄉鎮碼','year'],how='left')
final_6 = pd.merge(final_6,ind_per_6[['縣碼','high_per','mid_high_per','mid_low_per','year']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')

In [333]:
# 專利資料
patent_df = pd.read_excel(r'E:\paper\專利資料_matching對象.xlsx',index_col=0)
patent_df['縣碼'] = patent_df['縣碼'].astype(str)

In [334]:
final_6.drop(['patent stock','cite stock','claim stock'],axis=1,inplace=True)
final_9.drop(['patent stock','平均被引用','平均權利要求'],axis=1,inplace=True)
final_6 = pd.merge(final_6,patent_df[['縣碼','year','patent stock','cite stock','log patent stock','log cite stock','patent stock growth','cite stock growth']],left_on=['縣碼','year'],right_on=['縣碼','year'],how='left')

In [335]:
final_9 = final_9.set_index('鄉鎮碼')
final_6 = final_6.set_index('縣碼')
final_9['year'] = final_9['year'].astype(str)
final_6['year'] = final_6['year'].astype(str)

In [336]:
final_9[final_9['year']!='2010'].to_excel(r'E:\paper\新迴歸_9碼_lag2.xlsx')
final_6[final_6['year']!='2010'].to_excel(r'E:\paper\新迴歸_6碼_lag2.xlsx')